In [3]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, CLIPProcessor, CLIPModel
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torch import nn, optim
from sklearn.metrics import accuracy_score
from PIL import Image
import kagglehub

# Download and extract dataset using KaggleHub
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
print("Path to dataset files:", path)

data_dir = os.path.join(path, "chest_xray")

# Define Dataset and Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

def filter_mac_os_files(path):
    """Remove any macOS system files like .DS_Store or __MACOSX."""
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.startswith("._") or name == ".DS_Store":
                os.remove(os.path.join(root, name))
        if "__MACOSX" in dirs:
            os.rmdir(os.path.join(root, "__MACOSX"))

filter_mac_os_files(data_dir)

# Check for valid image extensions
valid_extensions = [".jpg", ".jpeg", ".png", ".ppm", ".bmp", ".pgm", ".tif", ".tiff", ".webp"]
def is_valid_image(file):
    return any(file.lower().endswith(ext) for ext in valid_extensions)

def filter_invalid_images(path):
    """Remove files that are not valid image types."""
    for root, dirs, files in os.walk(path):
        for name in files:
            if not is_valid_image(name):
                os.remove(os.path.join(root, name))

filter_invalid_images(data_dir)

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Step 2: Fine-Tune a Simple CNN for Image Classification
model = resnet18(weights="ResNet18_Weights.IMAGENET1K_V1")
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataset.classes))  # Adjust output layer for classification

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader)}")

train_model(model, dataloader, criterion, optimizer)

# Step 3: Load Pre-Trained CLIP for Vision-Language Tasks
clip_model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)
clip_model = CLIPModel.from_pretrained(clip_model_name)

# Step 4: Load LLM for Text Reasoning
llm_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer_text = AutoTokenizer.from_pretrained(llm_name)
model_text = AutoModelForSequenceClassification.from_pretrained(llm_name, num_labels=2)

# Step 5: Define Hierarchical Diagnosis Pipeline
def hierarchical_diagnosis(image_path, clinical_notes):
    # Image Analysis (Agent)
    image = Image.open(image_path).convert("RGB")
    image_inputs = clip_processor(images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        image_features = clip_model.get_image_features(**image_inputs)
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

    # Text Analysis (Manager)
    inputs = tokenizer_text(clinical_notes, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        text_output = model_text(**inputs)
        text_prediction = torch.argmax(text_output.logits, dim=1).item()
        text_result = "Disease Detected" if text_prediction == 1 else "No Disease Detected"

    # Combine Results
    if text_result == "Disease Detected":
        final_diagnosis = "High likelihood of disease based on clinical notes."
    else:
        final_diagnosis = "Further analysis needed for final determination."

    return {
        "Image Diagnosis": "Features Extracted",
        "Text Diagnosis": text_result,
        "Final Diagnosis": final_diagnosis
    }

# Step 6: Evaluate the Pipeline
image_path = os.path.join(data_dir, "train", "PNEUMONIA", os.listdir(os.path.join(data_dir, "train", "PNEUMONIA"))[0])  # Example image from the dataset
clinical_notes = "The patient exhibits symptoms of chronic cough and fever."
result = hierarchical_diagnosis(image_path, clinical_notes)

print("Pipeline Output:")
print(result)


Path to dataset files: /root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2


OSError: [Errno 39] Directory not empty: '/root/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/__MACOSX'